# Library Imports

In [12]:
import re
import pandas as pd
import math

# Helper Functions

### IGC parsing

In [32]:
def parse_igc(file_path):
    """
    Parses an IGC (International Gliding Commission) file and extracts flight data into a structured format.

    This function processes the IGC file, extracting `B` records that contain flight fixes. Each fix includes 
    the timestamp, latitude, longitude, GPS altitude, and barometric altitude. The latitude and longitude 
    values are converted from the IGC format (DDDMMmmm or DDMMmmm) to decimal degrees.

    Parameters:
    ----------
    file_path : str
        The file path to the IGC file to be parsed.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the parsed flight data with the following columns:
        - `time` (str): The UTC time of the fix in HHMMSS format.
        - `latitude` (float): The latitude of the fix in decimal degrees.
        - `longitude` (float): The longitude of the fix in decimal degrees.
        - `gps_altitude` (str): The GPS altitude of the fix in meters.
        - `baro_altitude` (str): The barometric altitude of the fix in meters.

    Notes:
    -----
    - The function contains a nested helper function `igc_to_decimal` for converting IGC coordinates 
      (DDDMMmmm or DDMMmmm) to decimal degrees.
    - This function assumes that the IGC file follows the standard format and contains valid `B` records.

    Raises:
    ------
    ValueError:
        If the IGC coordinate format is invalid (not 7 or 8 characters long).
    FileNotFoundError:
        If the specified file cannot be found or opened.

    Example:
    --------
    >>> parse_igc("flight.igc")
         time    latitude   longitude gps_altitude baro_altitude
    0  123456  46.658233    8.054550         1985          2133
    1  123457  46.658500    8.054700         1987          2135

    """
    
    # Convert IGC DDDMMmmm or DDMMmmm format to decimal degrees.
    def igc_to_decimal(igc_coordinate, direction):
        # Extract degrees, minutes, and thousandths of a minute
        if len(igc_coordinate) == 7:  # Latitude: DDMMmmm
            degrees = int(igc_coordinate[:2])
            minutes = int(igc_coordinate[2:4])
            thousandths = int(igc_coordinate[4:])
        elif len(igc_coordinate) == 8:  # Longitude: DDDMMmmm
            degrees = int(igc_coordinate[:3])
            minutes = int(igc_coordinate[3:5])
            thousandths = int(igc_coordinate[5:])
        else:
            raise ValueError("Invalid IGC coordinate format. Must be 7 or 8 characters.")

        # Convert to decimal degrees
        decimal_degrees = degrees + (minutes + thousandths / 1000) / 60

        # Apply negative sign for South or West directions
        if direction in ['S', 'W']:
            decimal_degrees *= -1

        return decimal_degrees
    
    # Open IGC file
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Read each B record in the IGC file and extract the corresponding information
    fixes = []
    for line in lines:
        if line.startswith("B"):
            timestamp = line[1:7]
            lat = igc_to_decimal(line[7:14], line[14:15])
            lon = igc_to_decimal(line[15:23], line[23:24])
            gps_alt = line[25:30]
            bar_alt = line[30:35]
            
            fixes.append([timestamp, lat, lon, gps_alt, bar_alt])
    
    return pd.DataFrame(fixes, columns=["time", 
                                        "latitude", 
                                        "longitude",  
                                        "gps_altitude", 
                                        "baro_altitude"
                                        ])

### Heading calculation between two points

In [14]:
def calculate_heading(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Difference in longitude
    delta_lon = lon2 - lon1
    
    # Calculate initial bearing
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    initial_bearing = math.atan2(x, y)
    
    # Convert from radians to degrees
    initial_bearing = math.degrees(initial_bearing)
    
    # Normalize to 0-360
    compass_bearing = (initial_bearing + 360) % 360
    
    return compass_bearing

### Placeholders

In [ ]:
# TODO: heading calculation between each pair of points

In [31]:
# TODO: configurable calculation for calculation of time taken to complete a turn

In [ ]:
# TODO: configurable calculation for glide ratio based on rate of change of heading  

In [ ]:
# TODO: basic sanity check visualization for climbing, gliding, thermalling

# Exploration

In [33]:
df = parse_igc("../data/raw/igc/rmfalquier.2024-08-31.10-43-13.IGC")
df.head()

,time,latitude,longitude,gps_altitude,baro_altitude
0,104300,46.658233,8.05455,01985,02133
1,104301,46.658233,8.05455,01985,02133
2,104302,46.658233,8.05455,01985,02133
3,104303,46.658233,8.05455,01985,02133
4,104304,46.658233,8.05455,01985,02133


In [34]:
# 104621 4639459N 00803331E
# 104622 4639463N 00803324E
lat1, lon1 = 46.39459, 8.03331   # Point 1: Latitude, Longitude
lat2, lon2 = 46.39463, 8.03324   # Point 2: Latitude, Longitude

heading = calculate_heading(lat1, lon1, lat2, lon2)
print(f"Heading: {heading:.2f}°")

Heading: 309.64°
